In [1]:
import pickle
from sklearn.pipeline import Pipeline
import logging
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(filename="5.model_build.log", level=logging.INFO, format='%(asctime)s %(message)s',datefmt="%Y-%m-%d %H:%M:%S")
features = ['TSH', 'TT4', 'T4U', 'FTI']
try:
    logging.info("model build started")
    try:
        x_train =  pd.read_csv("x_train.csv")
        y_train =  pd.read_csv("y_train.csv")
        logging.info("Sucessfully loading the train dataset========================")
    except Exception as e:
        logging.info(f"The error raise when loading the train dataset:{e}========================")
    try:
        x_test =  pd.read_csv("x_test.csv")
        y_test =  pd.read_csv("y_test.csv")
        logging.info("Sucessfully loading the test dataset========================")
    except Exception as e:
        logging.info(f"The error raise when loading the test dataset:{e}========================")
    try:
        pipe4 = Pipeline([("XGboost", XGBClassifier())])
        pipe6 = Pipeline([("random_forest", RandomForestClassifier())])
        logging.info("Sucessfully created pipeline=================================")
    except Exception as e:
        logging.info(f"The error raise when  created pipeline:{e}========================")
        
    def finding_best_parameter_rf(model, x, y, final_features):
        try:
            parameters= { 
                "random_forest__criterion":["gini","entropy"],
                "random_forest__n_estimators": [10, 100, 1000],
                "random_forest__max_depth":[5,8,15,25,30,None],
                "random_forest__min_samples_leaf":[1,2,5,10,15,100],
                "random_forest__max_leaf_nodes": [2, 5,10]
            }
            best_parameter_model = RandomizedSearchCV(model, param_distributions=parameters, cv=3)
            best_parameter_model.fit(x[final_features], y)
            logging.info(best_parameter_model.best_params_)
            logging.info(best_parameter_model.best_score_)
            return best_parameter_model
        except Exception as e:
            logging.info(f"The error raise when fit the dataset:{e}========================")
    random_forest =finding_best_parameter_rf(pipe6, x_train, y_train, features)
    features = ["FTI", "TSH", "TT4", "T4U"] 
    def finding_best_parameter_xb(model, x, y, final_features):
        try:
            parameters= { 
                'XGboost__max_depth': [3,4,5,7,10,15,],
                'XGboost__learning_rate': [0.001, 0.0003, 0.005],
                'XGboost__n_estimators': [100, 150, 80, 200],
                'XGboost__colsample_bytree': [0.3, 0.5, 0.7, 0.9]
            }
            best_parameter_model = RandomizedSearchCV(model, param_distributions=parameters, cv=5)
            best_parameter_model.fit(x[final_features], y)
            logging.info(best_parameter_model.best_params_)
            logging.info(best_parameter_model.best_score_)
            logging.info("model build completed")
            return best_parameter_model
        except Exception as e:
            logging.info(f"The error raise when fit the dataset:{e}========================")
    xgboost = finding_best_parameter_xb(pipe4, x_train, y_train, features)
    try:
        pickle.dump(random_forest , open('RandomForestClassifier.pkl','wb'))
        logging.info("Successfully saved Random forest")
    except Exception as e:
            logging.info(f"The error raise when saved Random forest:{e}========================")
except Exception as e:
    logging.info(f"model build NOT completed:{e}========================")